In [1]:
#全局变量
hub_token = open('/root/hub_token.txt').read().strip()
repo_id = 'lansinuote/nlp.7.translation'
push_to_hub = True

In [2]:
from transformers import AutoTokenizer

#加载编码器
tokenizer = AutoTokenizer.from_pretrained('Helsinki-NLP/opus-mt-en-ro',
                                          use_fast=True)

print(tokenizer)

#编码试算
tokenizer.batch_encode_plus(
    [['Hello, this one sentence!', 'This is another sentence.']])

MarianTokenizer(name_or_path='Helsinki-NLP/opus-mt-en-ro', vocab_size=59543, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'})


/root/anaconda3/envs/pt39/lib/python3.9/site-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


{'input_ids': [[125, 778, 3, 63, 141, 9191, 23, 187, 32, 716, 9191, 2, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [3]:
from datasets import load_dataset


def get_dataset():
    #加载数据
    dataset = load_dataset(path='wmt16', name='ro-en')

    #采样,数据量太大了跑不动
    dataset['train'] = dataset['train'].shuffle(1).select(range(20000))
    dataset['validation'] = dataset['validation'].shuffle(1).select(range(200))
    dataset['test'] = dataset['test'].shuffle(1).select(range(200))

    #数据预处理
    def preprocess_function(data):
        #取出数据中的en和ro
        en = [ex['en'] for ex in data['translation']]
        ro = [ex['ro'] for ex in data['translation']]

        #源语言直接编码就行了
        data = tokenizer.batch_encode_plus(en, max_length=128, truncation=True)

        #目标语言在特殊模块中编码
        with tokenizer.as_target_tokenizer():
            data['labels'] = tokenizer.batch_encode_plus(
                ro, max_length=128, truncation=True)['input_ids']

        return data

    dataset = dataset.map(function=preprocess_function,
                          batched=True,
                          batch_size=1000,
                          num_proc=4,
                          remove_columns=['translation'])

    return dataset


if push_to_hub:
    dataset = get_dataset()
    dataset.push_to_hub(repo_id=repo_id, token=hub_token)

#直接使用我处理好的数据集
dataset = load_dataset(path=repo_id)

print(dataset, dataset['train'][0])

Found cached dataset wmt16 (/root/.cache/huggingface/datasets/wmt16/ro-en/1.0.0/746749a11d25c02058042da7502d973ff410e73457f3d305fc1177dc0e8c4227)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/wmt16/ro-en/1.0.0/746749a11d25c02058042da7502d973ff410e73457f3d305fc1177dc0e8c4227/cache-e153bf661ff30a9d.arrow
Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/wmt16/ro-en/1.0.0/746749a11d25c02058042da7502d973ff410e73457f3d305fc1177dc0e8c4227/cache-04de8b997cf89dc0.arrow
Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/wmt16/ro-en/1.0.0/746749a11d25c02058042da7502d973ff410e73457f3d305fc1177dc0e8c4227/cache-d85cda4b0f38d63d.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/wmt16/ro-en/1.0.0/746749a11d25c02058042da7502d973ff410e73457f3d305fc1177dc0e8c4227/cache-eac92af555fe1778.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/wmt16/ro-en/1.0.0/746749a11d25c02058042da7502d973ff410e73457f3d305fc1177dc0e8c4227/cache-6dc27e6d44541abb.arrow


...

Generating test split:   0%|          | 0/200 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/20000 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/lansinuote___parquet/lansinuote--nlp.7.translation-9ebe5b45a0781875/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 200
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 200
    })
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 20000
    })
}) {'input_ids': [460, 354, 3794, 12, 10677, 20, 5046, 14, 4, 2546, 37, 8, 397, 5551, 30, 10113, 37, 3501, 19814, 18, 8465, 20, 4, 44690, 782, 2, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [902, 576, 2946, 76, 10815, 17, 5098, 14997, 5, 559, 1140, 43, 2434, 6624, 27, 50, 337, 19216, 46, 22174, 17, 2317, 121, 16825, 2, 0]}


In [4]:
#这个函数和下面这个工具类等价,但我也是模仿实现的,不确定有没有出入
#from transformers import DataCollatorForSeq2Seq
#DataCollatorForSeq2Seq(tokenizer, model=model)

import torch


#数据批处理函数
def collate_fn(data):
    #求最长的label
    max_length = max([len(i['labels']) for i in data])

    #把所有的label都补pad到最长
    for i in data:
        pads = [-100] * (max_length - len(i['labels']))
        i['labels'] = i['labels'] + pads

    #把多个数据整合成一个tensor
    data = tokenizer.pad(
        encoded_inputs=data,
        padding=True,
        max_length=None,
        pad_to_multiple_of=None,
        return_tensors='pt',
    )

    #定义decoder_input_ids
    data['decoder_input_ids'] = torch.full_like(data['labels'],
                                                tokenizer.get_vocab()['<pad>'],
                                                dtype=torch.long)
    data['decoder_input_ids'][:, 1:] = data['labels'][:, :-1]
    data['decoder_input_ids'][data['decoder_input_ids'] ==
                              -100] = tokenizer.get_vocab()['<pad>']

    return data


data = [{
    'input_ids': [21603, 10, 37, 3719, 13],
    'attention_mask': [1, 1, 1, 1, 1],
    'labels': [10455, 120, 80]
}, {
    'input_ids': [21603, 10, 7086, 8408, 563],
    'attention_mask': [1, 1, 1, 1, 1],
    'labels': [301, 53, 4074, 1669]
}]

collate_fn(data)['decoder_input_ids']

tensor([[59542, 10455,   120,    80],
        [59542,   301,    53,  4074]])

In [5]:
import torch

#数据加载器
loader = torch.utils.data.DataLoader(
    dataset=dataset['train'],
    batch_size=8,
    collate_fn=collate_fn,
    shuffle=True,
    drop_last=True,
)

for i, data in enumerate(loader):
    break

for k, v in data.items():
    print(k, v.shape, v[:2])

len(loader)

input_ids torch.Size([8, 38]) tensor([[   68,   346,    32,  1305,    13,   408,    45,  4682,  8355, 21463,
           101,  2868,  5941,     4,   530,   143,   246,     2,     0, 59542,
         59542, 59542, 59542, 59542, 59542, 59542, 59542, 59542, 59542, 59542,
         59542, 59542, 59542, 59542, 59542, 59542, 59542, 59542],
        [  115,    64, 10096,  2649,   131,     4,  1824, 36635,    20, 22789,
           433,     4, 24505,     2,     0, 59542, 59542, 59542, 59542, 59542,
         59542, 59542, 59542, 59542, 59542, 59542, 59542, 59542, 59542, 59542,
         59542, 59542, 59542, 59542, 59542, 59542, 59542, 59542]])
attention_mask torch.Size([8, 38]) tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
labels torch.Size([8, 45]) tensor([[  694,    55,  9685,    1

2500

In [6]:
from transformers import AutoModelForSeq2SeqLM, MarianModel, PreTrainedModel, PretrainedConfig

#加载模型
#model = AutoModelForSeq2SeqLM.from_pretrained('Helsinki-NLP/opus-mt-en-ro')


#定义下游任务模型
class Model(PreTrainedModel):
    config_class = PretrainedConfig

    def __init__(self, config):
        super().__init__(config)
        self.pretrained = MarianModel.from_pretrained(
            'Helsinki-NLP/opus-mt-en-ro')

        self.register_buffer('final_logits_bias',
                             torch.zeros(1, tokenizer.vocab_size))

        self.fc = torch.nn.Linear(512, tokenizer.vocab_size, bias=False)

        #加载预训练模型的参数
        parameters = AutoModelForSeq2SeqLM.from_pretrained(
            'Helsinki-NLP/opus-mt-en-ro')
        self.fc.load_state_dict(parameters.lm_head.state_dict())

        self.criterion = torch.nn.CrossEntropyLoss()

    def forward(self, input_ids, attention_mask, labels, decoder_input_ids):
        logits = self.pretrained(input_ids=input_ids,
                                 attention_mask=attention_mask,
                                 decoder_input_ids=decoder_input_ids)
        logits = logits.last_hidden_state

        logits = self.fc(logits) + self.final_logits_bias

        loss = self.criterion(logits.flatten(end_dim=1), labels.flatten())

        return {'loss': loss, 'logits': logits}


model = Model(PretrainedConfig())

#统计参数量
print(sum(i.numel() for i in model.parameters()) / 10000)

out = model(**data)

out['loss'], out['logits'].shape

Some weights of the model checkpoint at Helsinki-NLP/opus-mt-en-ro were not used when initializing MarianModel: ['final_logits_bias']
- This IS expected if you are initializing MarianModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MarianModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


10563.4816


(tensor(1.3505, grad_fn=<NllLossBackward0>), torch.Size([8, 45, 59543]))

In [7]:
from datasets import load_metric

#加载评价函数
metric = load_metric(path='sacrebleu')

#试算
metric.compute(predictions=['hello there', 'general kenobi'],
               references=[['hello there'], ['general kenobi']])

/tmp/ipykernel_26234/1560569803.py:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric(path='sacrebleu')


{'score': 0.0,
 'counts': [4, 2, 0, 0],
 'totals': [4, 2, 0, 0],
 'precisions': [100.0, 100.0, 0.0, 0.0],
 'bp': 1.0,
 'sys_len': 4,
 'ref_len': 4}

In [8]:
#测试
def test():
    model.eval()

    #数据加载器
    loader_test = torch.utils.data.DataLoader(
        dataset=dataset['test'],
        batch_size=8,
        collate_fn=collate_fn,
        shuffle=True,
        drop_last=True,
    )

    predictions = []
    references = []
    for i, data in enumerate(loader_test):
        #计算
        with torch.no_grad():
            out = model(**data)

        pred = tokenizer.batch_decode(out['logits'].argmax(dim=2))
        label = tokenizer.batch_decode(data['decoder_input_ids'])
        predictions.extend(pred)
        references.extend(label)

        if i % 2 == 0:
            print(i)
            input_ids = tokenizer.decode(data['input_ids'][0])

            print('input_ids=', input_ids)
            print('pred=', pred[0])
            print('label=', label[0])

        if i == 10:
            break

    references = [[j] for j in references]
    metric_out = metric.compute(predictions=predictions, references=references)
    print(metric_out)


test()

0
input_ids= You are a▁great▁power▁only▁if you▁have▁solutions.</s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
pred= Eşt tu o, doar daca ai solu soluțiitii.</s>,,,,,-- - - - Tu Tu Tu Tu Tu Tu Tu Tu Tu Tu Tu Tu Tu Tu Tu Tu Tu Tu Tu Tu Tu Tu Tu Tu Tu Tu Tu Tu Tu Tu Tu Tu Tu Tu Tu Tu Tu Tu Tu Tu Tu
label= <pad> Dar esti mare putere doar daca ai și solutii.</s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
2
input_ids= I▁can▁guarantee▁we▁haven't▁changed▁nothing▁all▁year with▁our▁defe

In [9]:
from transformers import AdamW
from transformers.optimization import get_scheduler


#训练
def train():
    optimizer = AdamW(model.parameters(), lr=2e-5)
    scheduler = get_scheduler(name='linear',
                              num_warmup_steps=0,
                              num_training_steps=len(loader),
                              optimizer=optimizer)

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.train()
    model.to(device)
    for i, data in enumerate(loader):
        for k in data.keys():
            data[k] = data[k].to(device)
    
        out = model(**data)
        loss = out['loss']

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

        optimizer.zero_grad()
        model.zero_grad()

        if i % 50 == 0:
            out = out['logits'].argmax(dim=2)
            correct = (data['decoder_input_ids'] == out).sum().item()
            total = data['decoder_input_ids'].shape[1] * 8
            accuracy = correct / total
            del correct
            del total

            predictions = []
            references = []
            for j in range(8):
                pred = tokenizer.decode(out[j])
                label = tokenizer.decode(data['decoder_input_ids'][j])
                predictions.append(pred)
                references.append([label])

            metric_out = metric.compute(predictions=predictions,
                                        references=references)

            lr = optimizer.state_dict()['param_groups'][0]['lr']

            print(i, loss.item(), accuracy, metric_out, lr)

    model.to('cpu')


if push_to_hub:
    train()
    model.push_to_hub(repo_id=repo_id, use_auth_token=hub_token)

/root/anaconda3/envs/pt39/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


0 1.4797824621200562 0.0 {'score': 9.120008577043292, 'counts': [221, 179, 149, 124], 'totals': [588, 580, 572, 564], 'precisions': [37.585034013605444, 30.862068965517242, 26.04895104895105, 21.98581560283688], 'bp': 0.31944937686506625, 'sys_len': 588, 'ref_len': 1259} 1.9992e-05
50 1.293352723121643 0.0 {'score': 7.44164783389345, 'counts': [211, 153, 114, 87], 'totals': [578, 570, 562, 554], 'precisions': [36.50519031141869, 26.842105263157894, 20.284697508896798, 15.703971119133573], 'bp': 0.31483413242798336, 'sys_len': 578, 'ref_len': 1246} 1.9592e-05
100 0.885346531867981 0.0033783783783783786 {'score': 6.751734803815611, 'counts': [201, 142, 104, 75], 'totals': [573, 565, 557, 549], 'precisions': [35.07853403141361, 25.13274336283186, 18.671454219030522, 13.66120218579235], 'bp': 0.31004753070168034, 'sys_len': 573, 'ref_len': 1244} 1.9192000000000002e-05
150 0.8533743619918823 0.0 {'score': 13.686218267242388, 'counts': [171, 123, 94, 70], 'totals': [352, 344, 336, 328], 'pre

1400 0.5198501944541931 0.004901960784313725 {'score': 10.956261133157897, 'counts': [168, 137, 117, 101], 'totals': [411, 403, 395, 387], 'precisions': [40.87591240875913, 33.99503722084367, 29.620253164556964, 26.098191214470283], 'bp': 0.34032341912768727, 'sys_len': 411, 'ref_len': 854} 8.792e-06
1450 1.1050782203674316 0.0 {'score': 13.610779822702535, 'counts': [215, 156, 118, 87], 'totals': [472, 464, 456, 448], 'precisions': [45.55084745762712, 33.62068965517241, 25.87719298245614, 19.419642857142858], 'bp': 0.45953398785509414, 'sys_len': 472, 'ref_len': 839} 8.392e-06
1500 0.7206818461418152 0.0055147058823529415 {'score': 11.547276583069706, 'counts': [219, 178, 151, 128], 'totals': [521, 513, 505, 497], 'precisions': [42.034548944337814, 34.69785575048733, 29.900990099009903, 25.75452716297787], 'bp': 0.35470517407082913, 'sys_len': 521, 'ref_len': 1061} 7.992e-06
1550 0.9179471135139465 0.002976190476190476 {'score': 14.694674582896523, 'counts': [162, 117, 94, 76], 'total

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/423M [00:00<?, ?B/s]

In [10]:
#直接使用我训练好的模型
model = Model.from_pretrained(repo_id)

test()

Some weights of the model checkpoint at Helsinki-NLP/opus-mt-en-ro were not used when initializing MarianModel: ['final_logits_bias']
- This IS expected if you are initializing MarianModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MarianModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


0
input_ids= ▁After the public emotion▁passes,▁we▁will return to the▁reasons▁we▁have▁created▁this▁space, the▁freedom of▁movement that is not▁only for▁people but▁also for▁goods.</s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
pred= Dupăa ce emo emotia publica, ne vom intoarce la motivele pentru care am creat acesttiul,, libertatea libera de miscare care nu este numai pentru oameni, ci si pentru bunuri.</s> </s>,,,,,, - - - - - - - După După După După După După După După După După
label= <pad> Dupa ce trece emotia publica, ne vom intoarce la motivele pentru care am creat spatiul acesta, aceasta libertate de miscare care nu e numai pentru oameni, ci și pentru bunuri.</s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
2
input_ids= Am I▁bad for the▁industry?</s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>